<a href="https://colab.research.google.com/github/AhmedMorra1/Coursera_Capstone/blob/master/Dubai_Communities_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Where to open a Coworking Space in Dubai?**

Let's Start by importing the needed libraries

In [1]:
#handle arrays
import numpy as np
#handle the data analysis
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
#handle json files
import json
#convert json to dataframes
from pandas.io.json import json_normalize
#handle requests from api and wikipedia
import requests
#handle html pages
from bs4 import BeautifulSoup
#import visualization library
import matplotlib.cm as cm
import matplotlib.colors as colors
#import k means for clustering
from sklearn.cluster import KMeans
#install geopy to get longitude and latitude
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 
#install folium map rendering library
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print("Libraries imported.")

Libraries imported.


Neighborhoods in dubai are called communities and we can find a list of them here https://en.wikipedia.org/wiki/List_of_communities_in_Dubai 

So we need to extract the names of the communities from the wikipedia list, get their latitude and longitude and putting them into a data frame

Let's import the library needed to scrape table data

In [0]:
import lxml.html as lh
url = "https://en.wikipedia.org/wiki/List_of_communities_in_Dubai"
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]

Ok now we are good, let's continue

In [4]:
com_name = []
for i in range (1, 131):
  com_name.append(tr_elements[i][1].text_content())
print(com_name[0:5])  

['Abu Hail', 'Al Awir First', 'Al Awir Second', 'Al Bada', 'Al Baraha']


Now let's get their latitude and longitude using geopy library

In [5]:
address = 'Abu Hail, Dubai'
geolocator = Nominatim(user_agent='to_explore')
location = geolocator.geocode(address)
print(location.address)
print(location.latitude)
print(location.longitude)

Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة
25.28594185
55.3294435447813


In [0]:
com_names = []
com_lat = []
com_long = []
for i in com_name:
  try:
    address = '{}, Dubai'.format(i)
    geolocator = Nominatim(user_agent='to_explore')
    location = geolocator.geocode(address)
    name = location.address
    latitude = location.latitude
    longitude = location.longitude
    com_names.append(name)
    com_lat.append(latitude)
    com_long.append(longitude)
  except:
    continue  

In [7]:
print(len(com_names),len(com_lat),len(com_long))

50 50 50


Now we have 60 communities with their latitude and longitude

Let's add them to a dataframe

In [0]:
#first we make a dictionary out of the lists
com_dic = {'Name':com_names,'Latitude':com_lat,'Longitude':com_long}

In [54]:
#Now let's convert it to a data frame
comdf = pd.DataFrame.from_dict(com_dic)
comdf.rename(columns={'Name':'Neighborhood'},inplace=True)
comdf.head()

,Neighborhood,Latitude,Longitude
0,"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",25.285942,55.329444
1,"Al Baraha, Al Rigga, دبي, دبي‎, الإمارات العرب...",25.282576,55.320139
2,"American School of Dubai, شارع الخيل الأول, Al...",25.099977,55.182493
3,"Riggat Al Buteen, دبي, دبي‎, الإمارات العربية ...",25.263057,55.320584
4,"Al Garhoud, الجداف, دبي, دبي‎, 122374, الإمارا...",25.239831,55.355668


Let's get the latitude and longitude of Dubai

In [10]:
geolocator = Nominatim(user_agent='to_explore')
dlocation = geolocator.geocode('Dubai')
dlat = dlocation.latitude
dlong = dlocation.longitude
print(dlat,dlong)

25.0657 55.1713


Now let's visualize these *communities*

In [11]:
dubai_map = folium.Map([dlat,dlong])
for name,lat,lon in zip(comdf['Name'],comdf['Latitude'],comdf['Longitude']):
  label = '{}'.format(name)
  label = folium.Popup(label,parse_html=True)
  folium.CircleMarker([lat,lon],
                      popup=label,
                      color='blue',
                      fill=True,
                      fill_color='#3186cc',
                      fill_opacity=.7,
                      parse_html=False).add_to(dubai_map)
dubai_map                      

Let's check my foursquare API

In [12]:
CLIENT_ID = 'ABGD1FXT5SO01ODC3IGKVU2LCE1L5ZAX5IUFVZPAKX0RC5PX' # your Foursquare ID
CLIENT_SECRET = 'YKY2JNPHZ3Z2UGTHGDS313PLTNAUYROFMRUYSMENOPHIDHJJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ABGD1FXT5SO01ODC3IGKVU2LCE1L5ZAX5IUFVZPAKX0RC5PX
CLIENT_SECRET:YKY2JNPHZ3Z2UGTHGDS313PLTNAUYROFMRUYSMENOPHIDHJJ


let's check the venues for one of the communities

In [13]:
comdf.iloc[0,0]

'Abu Hail, دبي, دبي\u200e, الإمارات العربية المتحدة'

In [14]:
neighborhood_latitude = comdf.iloc[0, 1] # neighborhood latitude value
neighborhood_longitude = comdf.iloc[0, 2] # neighborhood longitude value

neighborhood_name = comdf.iloc[0, 0] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة are 25.28594185, 55.3294435447813.


In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ABGD1FXT5SO01ODC3IGKVU2LCE1L5ZAX5IUFVZPAKX0RC5PX&client_secret=YKY2JNPHZ3Z2UGTHGDS313PLTNAUYROFMRUYSMENOPHIDHJJ&v=20180605&ll=25.28594185,55.3294435447813&radius=500&limit=100'

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df73a87923935001c614d44'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4f917049e4b0c93d7f58fc4d-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'primary': True,
         'shortName': 'Park'}],
       'id': '4f917049e4b0c93d7f58fc4d',
       'location': {'cc': 'AE',
        'city': 'دبي',
        'country': 'الإمارات العربية المتحدة',
        'distance': 358,
        'formattedAddress': ['دبي', 'الإمارات العربية المتحدة'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 25.285709645661722,
          'lng': 55.33299999592079}],
        'lat': 25.28570964566

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Hamriya Park,Park,25.285710,55.333000
1,Pond Park - Al Qusais,Park,25.288060,55.332606
2,Lively,Track,25.285194,55.325276
3,Jannati Health Club and Spa,Spa,25.285408,55.325168


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
dubai_venues = getNearbyVenues(names=comdf['Name'],
                                   latitudes=comdf['Latitude'],
                                   longitudes=comdf['Longitude']
                                  )

Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة
Al Baraha, Al Rigga, دبي, دبي‎, الإمارات العربية المتحدة
American School of Dubai, شارع الخيل الأول, Al Barsha, دبي‎, 6834, الإمارات العربية المتحدة
Riggat Al Buteen, دبي, دبي‎, الإمارات العربية المتحدة
Al Garhoud, الجداف, دبي, دبي‎, 122374, الإمارات العربية المتحدة
Al Hudaiba, دبي‎, الإمارات العربية المتحدة
الجافلية, شارع الشيخ خليفة بن زايد, Al Mankhool, دبي, دبي‎, 333840, United Arab Emirates
الكرامة, دبي, دبي‎, الإمارات العربية المتحدة
الممزر, دبي, دبي‎, 00000, الإمارات العربية المتحدة
Al Manara, القز, دبي, دبي‎, الإمارات العربية المتحدة
Al Mankhool, دبي, دبي‎, 111695, الإمارات العربية المتحدة
Green Digital, 2c شارع, نايف, دبي, دبي‎, 21024, الإمارات العربية المتحدة
Al Muraqqabat, دبي, دبي‎, الإمارات العربية المتحدة
Al Murar, دبي, دبي‎, 6073, الإمارات العربية المتحدة
Al Muteena, دبي, دبي‎, الإمارات العربية المتحدة
Comercical Bank of Dubai, شارع حمدان بن محمد, Al Markaziyah West, Al Markaziyah, أبوظبي, أبو ظبي, 44548, United Arab Emirates

In [21]:
print(dubai_venues.shape)
dubai_venues.head()

(796, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",25.285942,55.329444,Hamriya Park,25.285710,55.333000,Park
1,"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",25.285942,55.329444,Pond Park - Al Qusais,25.288060,55.332606,Park
2,"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",25.285942,55.329444,Lively,25.285194,55.325276,Track
3,"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",25.285942,55.329444,Jannati Health Club and Spa,25.285408,55.325168,Spa
4,"Al Baraha, Al Rigga, دبي, دبي‎, الإمارات العرب...",25.282576,55.320139,Ewa Hotel,25.282863,55.316261,Hotel


In [22]:
dubai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",4,4,4,4,4,4
"Al Baraha, Al Rigga, دبي, دبي‎, الإمارات العربية المتحدة",4,4,4,4,4,4
"Al Garhoud, الجداف, دبي, دبي‎, 122374, الإمارات العربية المتحدة",10,10,10,10,10,10
"Al Hudaiba, دبي‎, الإمارات العربية المتحدة",47,47,47,47,47,47
"Al Manara, القز, دبي, دبي‎, الإمارات العربية المتحدة",3,3,3,3,3,3
"Al Mankhool, دبي, دبي‎, 111695, الإمارات العربية المتحدة",20,20,20,20,20,20
"Al Muraqqabat, دبي, دبي‎, الإمارات العربية المتحدة",38,38,38,38,38,38
"Al Murar, دبي, دبي‎, 6073, الإمارات العربية المتحدة",16,16,16,16,16,16
"Al Muteena, دبي, دبي‎, الإمارات العربية المتحدة",27,27,27,27,27,27


In [23]:
print('There are {} uniques categories.'.format(len(dubai_venues['Venue Category'].unique())))

There are 143 uniques categories.


In [24]:
# one hot encoding
dubai_onehot = pd.get_dummies(dubai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dubai_onehot['Neighborhood'] = dubai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dubai_onehot.columns[-1]] + list(dubai_onehot.columns[:-1])
dubai_onehot = dubai_onehot[fixed_columns]

dubai_onehot.head()

,Neighborhood,African Restaurant,Airport Lounge,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Auto Garage,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Bed & Breakfast,Boat or Ferry,Breakfast Spot,Buffet,Burger Joint,Bus Station,Cafeteria,Café,Campground,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Convenience Store,Cupcake Shop,Currency Exchange,Department Store,Dessert Shop,Dim Sum Restaurant,Donut Shop,Electronics Store,English Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Iraqi Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Movie Theater,Music Store,Nightclub,North Indian Restaurant,Office,Organic Grocery,Outdoors & Recreation,Pakistani Restaurant,Park,Persian Restaurant,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Record Shop,Recreation Center,Resort,Restaurant,Roof Deck,Sandwich Place,Seafood Restaurant,Shawarma Place,Shipping Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tennis Court,Thai Restaurant,Theme Restaurant,Tour Provider,Track,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Water Park,Waterfront,Women's Store,Yoga Studio
0,"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Al Baraha, Al Rigga, دبي, دبي‎, الإمارات العرب...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
dubai_onehot.shape

(796, 144)

In [26]:
dubai_grouped = dubai_onehot.groupby('Neighborhood').mean().reset_index()
dubai_grouped

,Neighborhood,African Restaurant,Airport Lounge,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Auto Garage,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Bed & Breakfast,Boat or Ferry,Breakfast Spot,Buffet,Burger Joint,Bus Station,Cafeteria,Café,Campground,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Convenience Store,Cupcake Shop,Currency Exchange,Department Store,Dessert Shop,Dim Sum Restaurant,Donut Shop,Electronics Store,English Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Iraqi Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Movie Theater,Music Store,Nightclub,North Indian Restaurant,Office,Organic Grocery,Outdoors & Recreation,Pakistani Restaurant,Park,Persian Restaurant,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Record Shop,Recreation Center,Resort,Restaurant,Roof Deck,Sandwich Place,Seafood Restaurant,Shawarma Place,Shipping Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tennis Court,Thai Restaurant,Theme Restaurant,Tour Provider,Track,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Water Park,Waterfront,Women's Store,Yoga Studio
0,"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,"Al Baraha, Al Rigga, دبي, دبي‎, الإمارات العرب...",0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000

In [27]:
dubai_grouped.shape

(44, 144)

In [28]:
num_top_venues = 5

for hood in dubai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dubai_grouped[dubai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة----
   venue  freq
0   Park  0.50
1  Track  0.25
2    Spa  0.25
3   Pool  0.00
4  Plaza  0.00


----Al Baraha, Al Rigga, دبي, دبي‎, الإمارات العربية المتحدة----
                       venue  freq
0                       Café  0.25
1                Coffee Shop  0.25
2                      Hotel  0.25
3  Middle Eastern Restaurant  0.25
4         African Restaurant  0.00


----Al Garhoud, الجداف, دبي, دبي‎, 122374, الإمارات العربية المتحدة----
            venue  freq
0           Hotel   0.2
1      Restaurant   0.1
2  Airport Lounge   0.1
3            Café   0.1
4    Dessert Shop   0.1


----Al Hudaiba, دبي‎, الإمارات العربية المتحدة----
                       venue  freq
0                      Hotel  0.11
1           Asian Restaurant  0.11
2                       Café  0.09
3  Middle Eastern Restaurant  0.09
4         Chinese Restaurant  0.04


----Al Manara, القز, دبي, دبي‎, الإمارات العربية المتحدة----
             venue  freq
0     Dess

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dubai_grouped['Neighborhood']

for ind in np.arange(dubai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dubai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",Park,Track,Spa,Flea Market,Gift Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop
1,"Al Baraha, Al Rigga, دبي, دبي‎, الإمارات العرب...",Hotel,Middle Eastern Restaurant,Coffee Shop,Café,Flea Market,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop
2,"Al Garhoud, الجداف, دبي, دبي‎, 122374, الإمارا...",Hotel,Pool,Airport Lounge,Dessert Shop,Café,Basketball Court,Restaurant,Coffee Shop,Shipping Store,Flea Market
3,"Al Hudaiba, دبي‎, الإمارات العربية المتحدة",Hotel,Asian Restaurant,Café,Middle Eastern Restaurant,Convenience Store,Chinese Restaurant,Bar,Fried Chicken Joint,Smoke Shop,Coffee Shop
4,"Al Manara, القز, دبي, دبي‎, الإمارات العربية ا...",Dessert Shop,Gym,Playground,Flea Market,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Fish Market


In [46]:
# set number of clusters
kclusters = 5

dubai_grouped_clustering = dubai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dubai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 3,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 0, 0, 0, 0],
      dtype=int32)

In [47]:
comdf.rename(columns={'Name':'Neighborhood'},inplace=True)
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()
# dubai_merged = comdf

# # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
# dubai_merged = dubai_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on=dubai_merged.iloc[:,2])



# # check the last columns!
# dubai_merged.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",Park,Track,Spa,Flea Market,Gift Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop
1,0,"Al Baraha, Al Rigga, دبي, دبي‎, الإمارات العرب...",Hotel,Middle Eastern Restaurant,Coffee Shop,Café,Flea Market,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop
2,0,"Al Garhoud, الجداف, دبي, دبي‎, 122374, الإمارا...",Hotel,Pool,Airport Lounge,Dessert Shop,Café,Basketball Court,Restaurant,Coffee Shop,Shipping Store,Flea Market
3,0,"Al Hudaiba, دبي‎, الإمارات العربية المتحدة",Hotel,Asian Restaurant,Café,Middle Eastern Restaurant,Convenience Store,Chinese Restaurant,Bar,Fried Chicken Joint,Smoke Shop,Coffee Shop
4,0,"Al Manara, القز, دبي, دبي‎, الإمارات العربية ا...",Dessert Shop,Gym,Playground,Flea Market,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Fish Market


In [55]:
dubai_merged = comdf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dubai_merged = dubai_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



# check the last columns!
dubai_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",25.285942,55.329444,0.0,Park,Track,Spa,Flea Market,Gift Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop
1,"Al Baraha, Al Rigga, دبي, دبي‎, الإمارات العرب...",25.282576,55.320139,0.0,Hotel,Middle Eastern Restaurant,Coffee Shop,Café,Flea Market,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop
2,"American School of Dubai, شارع الخيل الأول, Al...",25.099977,55.182493,0.0,Hotel,Bar,Nightclub,Café,Coffee Shop,Yoga Studio,Mexican Restaurant,Pharmacy,Organic Grocery,Middle Eastern Restaurant
3,"Riggat Al Buteen, دبي, دبي‎, الإمارات العربية ...",25.263057,55.320584,0.0,Hotel,Restaurant,Ice Cream Shop,Middle Eastern Restaurant,Turkish Restaurant,Mediterranean Restaurant,Juice Bar,Café,Pizza Place,Pier
4,"Al Garhoud, الجداف, دبي, دبي‎, 122374, الإمارا...",25.239831,55.355668,0.0,Hotel,Pool,Airport Lounge,Dessert Shop,Café,Basketball Court,Restaurant,Coffee Shop,Shipping Store,Flea Market


In [58]:
dubai_merged['Cluster Labels'].unique()

array([ 0.,  3., nan,  1.,  4.,  2.])

Let's remove the nan clusters

In [63]:
for index,row in dubai_merged.iterrows():
  if row['Cluster Labels'] not in [0.0,  3.0, 1.0,  4.0,  2.0]:
    dubai_merged.drop(index,inplace=True)
dubai_merged['Cluster Labels'].unique()    

array([0., 3., 1., 4., 2.])

Let's visualize the clusters

In [64]:
map_clusters = folium.Map(location=[dlat, dlong])

#set color scheme for the clusters
markers_colors = {0.0:'blue',1.0:'green',2.0:'yellow',3.0:'black',4.0:'pink'}
c=0
for lat, lon, poi, cluster in zip(dubai_merged['Latitude'], dubai_merged['Longitude'], dubai_merged['Neighborhood'], dubai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=markers_colors[cluster],
        fill=True,
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
    
       
map_clusters

Let's Explore the cluster

In [48]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 0, neighborhoods_venues_sorted.columns[[1] + list(range(5, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Abu Hail, دبي, دبي‎, الإمارات العربية المتحدة",Flea Market,Gift Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop
1,"Al Baraha, Al Rigga, دبي, دبي‎, الإمارات العرب...",Café,Flea Market,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop
2,"Al Garhoud, الجداف, دبي, دبي‎, 122374, الإمارا...",Dessert Shop,Café,Basketball Court,Restaurant,Coffee Shop,Shipping Store,Flea Market
3,"Al Hudaiba, دبي‎, الإمارات العربية المتحدة",Middle Eastern Restaurant,Convenience Store,Chinese Restaurant,Bar,Fried Chicken Joint,Smoke Shop,Coffee Shop
4,"Al Manara, القز, دبي, دبي‎, الإمارات العربية ا...",Flea Market,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Fish Market
5,"Al Mankhool, دبي, دبي‎, 111695, الإمارات العرب...",Asian Restaurant,Sports Bar,Roof Deck,Breakfast Spot,Indian Restaurant,Campground,Hookah Bar
6,"Al Muraqqabat, دبي, دبي‎, الإمارات العربية الم...",Fast Food Restaurant,Filipino Restaurant,Restaurant,Fried Chicken Joint,Grocery Store,Ice Cream Shop,Spa
7,"Al Murar, دبي, دبي‎, 6073, الإمارات العربية ال...",Café,Middle Eastern Restaurant,Asian Restaurant,Indie Movie Theater,Intersection,Smoke Shop,Mobile Phone Shop
8,"Al Muteena, دبي, دبي‎, الإمارات العربية المتحدة",Chinese Restaurant,Café,Iraqi Restaurant,Gym,Fast Food Restaurant,Clothing Store,Fried Chicken Joint
9,"Al Rashidiya, The Island District, دبي, دبي‎, ...",Café,Asian Restaurant,Fried Chicken Joint,Yoga Studio,French Restaurant,Gastropub,Furniture / Home Store


In [49]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 1, neighborhoods_venues_sorted.columns[[1] + list(range(2, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"Al Satwa, Downtown Burj Khalifa, دبي, دبي‎, 24...",Coffee Shop,Yoga Studio,Flea Market,Gift Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Fish Market
39,"جبل علي, قرية جبل علي, دبي‎, 14225, الإمارات ا...",Coffee Shop,Yoga Studio,Flea Market,Gift Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Fish Market


In [65]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 2, neighborhoods_venues_sorted.columns[[1] + list(range(2, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,"Wadi Alamardi, دبي‎, United Arab Emirates",Campground,Yoga Studio,Department Store,Gift Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Flea Market


In [66]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 3, neighborhoods_venues_sorted.columns[[1] + list(range(2, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,"Emarine, ATM drive through, ميناء جبل علي, دبي...",Restaurant,Yoga Studio,Flea Market,Gift Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Fish Market
38,"الممزر, دبي, دبي‎, 00000, الإمارات العربية الم...",Restaurant,Yoga Studio,Flea Market,Gift Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Fish Market


In [67]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 4, neighborhoods_venues_sorted.columns[[1] + list(range(2, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,"Dubai International Academy, شارع الخيل الأول,...",Tennis Court,Yoga Studio,Golf Course,Gift Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Flea Market
